In [2]:
!pip3 install ray

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import numpy as np
import ray
import time

In [4]:
# Start Ray.
ray.init()

@ray.remote
def f(x):
    time.sleep(1)
    return x

# Start 4 tasks in parallel.
result_ids = []
for i in range(4):
    result_ids.append(f.remote(i))
    
# Wait for the tasks to complete and retrieve the results.
# With at least 4 cores, this will take 1 second.
results = ray.get(result_ids)  # [0, 1, 2, 3]

In [14]:
import numpy as np
from joblib import parallel_backend # added line.
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC      

from ray.util.joblib import register_ray # added line.
register_ray() # added line.

param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}

model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=5, n_iter=300,verbose=1)
digits = load_digits()

In [16]:
#ray.init(address=”auto”)
def kobe():
    with parallel_backend('ray',n_jobs=-1): # added line.
        search.fit(digits.data, digits.target)

In [23]:
%timeit -n 1 -r 1 kobe()

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
44.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [22]:
search2 = RandomizedSearchCV(model, param_space, cv=5, n_iter=300,verbose=1,n_jobs=-1)
%timeit -n 1 -r 1 search2.fit(digits.data, digits.target)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
36.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [24]:
%timeit -n 1 -r 1 search.fit(digits.data, digits.target)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
4min 23s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
n = 10000
p = 40
rng = np.random.RandomState(8888)
x_mat = np.concatenate((np.full(n,1).reshape(-1,1),rng.normal(size=(n,p))),axis=1) 
beta_true = rng.normal(size = p+1)
y = np.dot(x_mat,beta_true) + rng.normal(n)